In [1]:
import csv
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
# reading archive_map csv data into people_Drehem.csv

archives = pd.read_csv("sumerian_archive_map.csv")
#archives[archives.PID=='P'+'109555'].Archive.iloc[0]
people = pd.read_csv("people_Drehem.csv")


archives = pd.read_csv("archive_map.csv")

archives['PID'] = [int(archives['PID'][i][1:]) for i in np.arange(len(archives['PID']))]
archives
# archives[archives['PID'] == 104219]
people = people.merge(archives, how='inner', left_on='p index', right_on='PID')
people.to_csv("people_Archives_Drehem.csv")

FileNotFoundError: File b'sumerian_archive_map.csv' does not exist

In [ ]:
# container to store row data
class Node:
    def __init__(self, name, norm_name, role, profession, family, p_index, year, processed, archive):
        self.name = name
        self.norm_name = norm_name
        self.role = role
        self.profession = profession
        self.family = family
        self.p_index = p_index
        self.year = year
        self.processed = processed
        self.archive = archive
        self.id = None

    def add_id(self, id):
        self.id = id


# traverses through people_Drehem.csv to create a node per row

def create_nodes_list():
    
    with open('people_Archives_Drehem.csv', 'rt', encoding="utf-8") as csvfile:

        reader = csv.DictReader(csvfile)
        node_list = list()
        for row in reader:
            name = row['name']
            norm_name = row['normalized name']
            role = row['roles']
            profession = row['profession']
            family = row['family']
            p_index = row['p index']
            year = row['date name']
            processed = row['processed date']
            archive = row['Archive']
            node = Node(name, norm_name, role, profession, family, p_index, year, processed, archive)
            node_list.append(node)
            
    return node_list

In [ ]:
def create_new_nodes_list(list_of_nodes):
    with open('attestation_nodes.csv', 'w') as csvfile:
        fieldnames = ['id', 'name','norm_name', 'role', 'profession', 'processed year', 'archive', 'family', 'p_index', 'date name']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        curr_id = 1

        for node in list_of_nodes:
            name = node.name
            if type(node) is list:
                node = node[0]

            role = node.role
            norm_name = node.norm_name
            profession = node.profession
            family = node.family
            p_index = node.p_index
            year = node.year
            archive = node.archive
            
            # if there is no processed year data, sets Max and MinYear to be 0
            # these nodes will be ignored in the final timeline visualization
            processed = node.processed
            if len(processed)>0:
                maxYear = max(processed)
                minYear = min(processed)
            else:
                maxYear = 0
                minYear = 0
            
            maxDiff = 0
            gStart = 0 # start year of largest gap between transactions for each node
            gEnd = 0 # end year of largest gap between transactions for each node
            pre_GapInstanceCount =  0
            post_GapInstanceCount = 0
            node.add_id(curr_id)
            
            #calculating proportions for Archives
            prop_doa = 100*np.count_nonzero(np.array(archive)=='domesticated animal')/len(archive)
            prop_dea = 100*np.count_nonzero(np.array(archive)=='dead animal')/len(archive)
            prop_lo = 100*np.count_nonzero(np.array(archive)=='leather object')/len(archive)
            prop_po = 100*np.count_nonzero(np.array(archive)=='precious object')/len(archive)
            prop_q = 100*np.count_nonzero(np.array(archive)=='queen')/len(archive)
            prop_s = 100*np.count_nonzero(np.array(archive)=='seal')/len(archive)
            prop_u = 100*np.count_nonzero(np.array(archive)=='unknown')/len(archive)
            prop_wa = 100*np.count_nonzero(np.array(archive)=='wild animal')/len(archive)
            prop_w = 100*np.count_nonzero(np.array(archive)=='wool')/len(archive)

            writer.writerow({
                'id': curr_id,
                'name': name,
                'norm_name':norm_name,
                'role': role,
                'profession': profession,
                'family': family,
                'p_index': p_index,
                'date name': year,
                'archive':archive,
                'processed year': processed,
                })
            curr_id += 1

        csvfile.close()
        
        #roles = ['source', 'intermediary', 'recipient', 'source', 'representative']
        #people_filt =  pd.read_csv('archive_proportions.csv')
        #people_roles = people_filt['role']
        #for r in roles:
        #    people_filt["% " + r] = people_roles.map(lambda a: a.count(r)/(a.count(",")+1))
        #people_filt.to_csv('new_nodes.csv', index = False)

Edges

In [ ]:
def fill_new_person_to_info(list_of_nodes):
    p_indexes_to_people = defaultdict(list)
    for node in list_of_nodes:
        role = node.role
        p_index = node.p_index
        year = node.year
        p_indexes_to_people[p_index].append(node)
        
    return p_indexes_to_people


def create_edge_list(id_to_people):
    with open('attestation_edges.csv', 'w') as csvfile:
        fieldnames = ['id', 'source', 'target', 'p_index', 'year', 'type', 'source\'s role', 'target\'s role']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        curr_id = 1
        for key, value in id_to_people.items():
            if len(value) == 2:
                list_of_roles = []
                role_to_node = {}
                node1 = None
                node2 = None

                for node in value:
                    #print(type(node.role))
                    list_of_roles.append(node.role)
                    role_to_node[node.role] = node
                    
               # print(list_of_roles)    
                if "{'source'}" in list_of_roles:
                    role1 = "{'source'}"
                    person1 = (role_to_node[role1]).name
                    node1 = role_to_node[role1]
                    id1 = (role_to_node[role1]).id

                    list_of_roles.remove("{'source'}")

                    role2 = list_of_roles[0]
                    person2 = (role_to_node[role2]).name
                    node2 = role_to_node[role2]
                    id2 = (role_to_node[role2]).id

                    edge_type = "Directed"

                # there are 456 transactions between other person -> "recipient"
                # thus this creates 456 edges
                # sometimes there are "recipients" -> "recipients" in a transactions, so the edge is chose arbitrarily
                elif "{'recipient'}" in list_of_roles:
                    role2 = "{'recipient'}"
                    node2 = role_to_node[role2]
                    person2 = node2.name
                    id2 = node2.id

                    list_of_roles.remove("{'recipient'}")

                    role1 = list_of_roles[0]
                    node1 = role_to_node[role1]
                    person1 = node1.name
                    id1 = node1.id

                    edge_type = "Directed"

                # there are 280 transactions where there are neither a source nor a recipient
                # thus this creates 280 edges
                else:
                    role1 = list_of_roles[0]
                    person1 = (role_to_node[role1]).name
                    node1 = (role_to_node[role1])
                    id1 = node1.id

                    role2 = list_of_roles[1]
                    person2 = (role_to_node[role2]).name
                    node2 = (role_to_node[role2])
                    id2 = node2.id

                    edge_type = "Undirected"

                edge_id = curr_id
                curr_id += 1
                p_index = key
                #index_of_year = node1.p_index.index(p_index)
                #modified 10/5
                year = node1.year
                writer.writerow({
                            'id': edge_id,
                            'source': id1,
                            'target': id2,
                            'p_index': p_index,
                            'year': year,
                            'type': edge_type,
                            'source\'s role': role1,
                            'target\'s role': role2
                            })
                
            elif len(value) == 3:
                list_of_roles = []
                role_to_node_list = defaultdict(list)
                node1 = None
                node2 = None
                node3 = None

                for node in value:
                    list_of_roles.append(node.role)
                    role_to_node_list[node.role].append(node)

                if "{'source'}" in list_of_roles and "{'recipient'}" in list_of_roles:
                    # source, recipient1, recipient2: source -> recipient1, source -> recipient2 (creates 2 edges)
                    # there are 196 transactions of this structure
                    # thus this creates 392 edges
                    #print("I am a p-index with more than 3 nodes associated with me.")
                    if list_of_roles.count("{'recipient'}") == 2:
                        role1 = "{'source'}"
                        #person1 = role_to_node_list[role1][0]
                        node1 = role_to_node_list[role1][0]
                        id1 = node1.id

                        role2 = "{'recipient'}"
                        #person2 = role_to_node_list[role2][0][0]
                        node2 = role_to_node_list[role2][0]
                        id2 = node2.id

                        role3 = "{'recipient'}"
                        #person3 = role_to_node_list[role2][1][0]
                        node3 = role_to_node_list[role2][1]
                        id3 = node3.id

                        edge_type = "Directed"
                        edge_id = curr_id
                        curr_id += 1
                        p_index = key
                        #index_of_year = node1.p_index.index(p_index)
                        year = node1.year

                        writer.writerow({
                            'id': edge_id,
                            'source': id1,
                            'target': id2,
                            'p_index': p_index,
                            'year': year,
                            'type': edge_type,
                            'source\'s role': role1,
                            'target\'s role': role2
                            })
                        edge_id = curr_id
                        curr_id += 1

                        writer.writerow({
                            'id': edge_id,
                            'source': id1,
                            'target': id2,
                            'p_index': p_index,
                            'year': year,
                            'type': edge_type,
                            'source\'s role': role1,
                            'target\'s role': role2
                            })
                    else:
                        role1 = "{'source'}"
                        node1 = role_to_node_list[role1][0]
                        id1 = node1.id

                        list_of_roles.remove("{'source'}")
                        list_of_roles.remove("{'recipient'}")

                        role2 = list_of_roles[0]
                        #person2 = role_to_node_list[role2][0][0]
                        node2 = role_to_node_list[role2][0]
                        id2 = node2.id

                        role3 = "{'recipient'}"
                        #person3 = role_to_node_list[role3][0][0]
                        node3 = role_to_node_list[role3][0]
                        id3 = node3.id

                        edge_type = "Directed"
                        edge_id = curr_id
                        curr_id += 1
                        p_index = key
                        #index_of_year = node1.p_index.index(p_index)
                        year = node1.year

                        writer.writerow({
                            'id': edge_id,
                            'source': id1,
                            'target': id2,
                            'p_index': p_index,
                            'year': year,
                            'type': edge_type,
                            'source\'s role': role1,
                            'target\'s role': role2
                            })
                        edge_id = curr_id
                        curr_id += 1
                        writer.writerow({
                            'id': edge_id,
                            'source': id1,
                            'target': id3,
                            'p_index': p_index,
                            'year': year,
                            'type': edge_type,
                            'source\'s role': role1,
                            'target\'s role': role3
                            })
                        edge_id = curr_id
                        curr_id += 1
                        writer.writerow({
                            'id': edge_id,
                            'source': id2,
                            'target': id3,
                            'p_index': p_index,
                            'year': year,
                            'type': edge_type,
                            'source\'s role': role2,
                            'target\'s role': role3
                            })
            elif len(value) > 3:
                list_of_roles = []
                node_list = []
                for node in value:
                    list_of_roles.append(node.role)
                    node_list.append(node)

                role1 = list_of_roles[0]
                node1 =  node_list[0]
                id1 = node1.id

                role2 = list_of_roles[1]
                node2 = node_list[1]
                id2 = node2.id

                role3 = list_of_roles[2]
                node3 =  node_list[2]
                id3 = node3.id

                edge_type = "Undirected"
                edge_id = curr_id
                curr_id += 1
                p_index = key
                index_of_year = node1.p_index
                year = node1.year

                writer.writerow({
                    'id': edge_id,
                    'source': id1,
                    'target': id2,
                    'p_index': p_index,
                    'year': year,
                    'type': edge_type,
                    'source\'s role': role1,
                    'target\'s role': role2
                    })
                edge_id = curr_id
                curr_id += 1
                writer.writerow({
                    'id': edge_id,
                    'source': id1,
                    'target': id3,
                    'p_index': p_index,
                    'year': year,
                    'type': edge_type,
                    'source\'s role': role1,
                    'target\'s role': role3
                    })
                edge_id = curr_id
                curr_id += 1
                writer.writerow({
                    'id': edge_id,
                    'source': id2,
                    'target': id3,
                    'p_index': p_index,
                    'year': year,
                    'type': edge_type,
                    'source\'s role': role2,
                    'target\'s role': role3
                    })


    csvfile.close()

In [ ]:
#merges attestation_nodes with commodities information 
#code from Commodities and Cohorts + DBSCAN.ipynb
def merge_with_commodities():
    node_dt = pd.read_csv("attestation_nodes.csv")
    com1 = pd.read_csv("commodities1.csv")
    com2 = pd.read_csv("commodities2.csv")
    commodities = pd.concat([com1[['ip','commodities']], com2[['ip','commodities']]])
    nodes_and_comm = pd.merge(node_dt, commodities, how = 'inner', left_on = 'p_index', right_on = 'ip').drop(['ip', 'id'], axis=1)
    nodes_and_comm.to_csv("attestation_nodes.csv")


In [ ]:
def main():
    node_list = create_nodes_list()
    create_new_nodes_list(node_list)
    id_people = fill_new_person_to_info(node_list)
    create_edge_list(id_people)
    #merge with commodities tables
    merge_with_commodities()
if __name__ == "__main__":
    main()